# Playing with Contrast Enhancement, Histogram Equalization
stough 202-

We've now seen how to improve image contrast through the application of transfer functions like [windowing, power/gamma, log, and others](./enhance_histeq.ipynb). We've also seen how [Histogram Equalization](./enhance_histeq.ipynb) creates an ideal transfer function in some sense--that of leading to a uniform histogram. Use these other demos in order to answer the following questions

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatLogSlider
from scipy.interpolate import interp1d

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)

from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       vis_pair,
                       lab_uniform)

## Find an Image

Find **at least** two reasonably inoffensive images (maybe a little offensive) to test on. One key here is that because we're doing contrast enhancement, you might err on the side of lower-contrast images. You don't have to do every test on both, but it wouldn't be difficult.

In [77]:
I1 = plt.imread('../dip_pics/roadforest.jpg')
I2 = plt.imread('../dip_pics/dark_nature.jpg')

## 1. Parametric Transfer Functions 

- Use `vis_hists` to **show the histogram of an image**. 
- With that visual assistance consider what parametric transfer function (power/gamma, log, exponent) might be helpful in this image's case.
- **Justify your choice** with written description
- **Display the result** of applying the transfer function of choice to the image. 
- You can use `vis_pair` to show both the original and the presumeably contrast-enhanced images together.
- **Explain how**, with written description, the figures you generate either do or do not reflect your justification from before. You can even **show a zoomed-in section** that illustrates your contentions.

In [78]:
vis_hists(I2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
I2_log_corrected = ((255/8)*np.log2(1.0 + I2)).astype('uint8')
vis_hists(I2_log_corrected)
vis_pair(I2, I2_log_corrected, second_title='Log Corrected', show_ticks=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2. Alternative Colorspaces

**Show** the same parametric contrast enhancement as in Part 1, but performed **only on Luminance/Y/Value** dimension after conversion into  [Lab](../Color/color_Lab.ipynb)/[YCbCr](../Color/color_YCbCr.ipynb)/[HSV](../Color/color_HSV.ipynb). **Try all three alternative colorspaces and explain your results**. Remember, you'll need to: 

- Convert to the alternative colorspace using the appropriate [skimage.color](https://scikit-image.org/docs/dev/api/skimage.color.html) function.
- Change only the one channel you're working with. Be sure you're not producing out-of-bounds values for the particular colorspace you choose.
- Convert back using the appropriate inverse from `skimage.color`.

You can use `vis_hists` and/or extend `vis_pair` to show all three together. **Compare and contrast your results**.

In [80]:
#Convert to alternative colorspace
I2_hsv = color.rgb2hsv(I2)
I2_lab = color.rgb2lab(I2)
I2_ybr = color.rgb2ycbcr(I2)

# LAB Space

In [81]:
vis_hists(I2_lab[...,0])
L,A,B = np.split(I2_lab, indices_or_sections=3,axis=-1)
temp = L.copy()
arr_info(temp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((2678, 4480, 1), dtype('float64'), 0.039573654069084085, 66.6368544813074)

In [82]:
arr_info((100/np.log2(68))*np.log2(1.0 + temp))

((2678, 4480, 1), dtype('float64'), 0.9197916866822682, 99.87309690073452)

In [83]:
arr_info(B)

((2678, 4480, 1), dtype('float64'), -40.13728101601954, 10.940806466672925)

In [84]:
L_e = (100/np.log2(68))*np.log2(1.0 + temp)
I2_lab_e = np.dstack((L_e,A,B))
I2_lab2rgb = color.lab2rgb(I2_lab_e)

In [85]:
arr_info(I2_lab2rgb[...,2])

((2678, 4480), dtype('float64'), 0.02043412630267046, 1.0)

In [86]:
vis_hists(I2_lab2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
vis_pair(I2,I2_lab2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
def applyGamma(I, gamma = 1):
    c = 100.0/(np.power(100.0, gamma))
    J = c*np.power(I.astype('float'), gamma)
    return J

In [89]:
J = L.copy()
arr_info(J)

((2678, 4480, 1), dtype('float64'), 0.039573654069084085, 66.6368544813074)

In [90]:
J = applyGamma(J,gamma = 0.3)

In [91]:
arr_info(J)

((2678, 4480, 1), dtype('float64'), 9.532829948570528, 88.53486849539097)

In [92]:
J2_e = np.dstack((J,A,B))
J2_f = (255*color.lab2rgb(J2_e)).astype('uint8')

In [93]:
vis_hists(J2_f)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
vis_pair(I2,J2_f)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# HSV Space

In [95]:
vis_hists(I2_hsv[...,2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
H,S,V = np.split(I2_hsv, indices_or_sections=3,axis=-1)
temp2 = V.copy()

In [97]:
arr_info(temp2)

((2678, 4480, 1), dtype('float64'), 0.00392156862745098, 0.7607843137254902)

In [98]:
arr_info((100/np.log2(76))*np.log2(1.0 + temp2)/13.07)

((2678, 4480, 1), dtype('float64'), 0.006914688477943239, 0.9995273987572477)

In [99]:
V_e = (100/np.log2(76))*np.log2(1.0 + temp2)/13.07

In [100]:
I2_hsv_e = np.dstack((H,S,V_e))
I2_hsv2rgb = color.hsv2rgb(I2_hsv_e)

In [101]:
vis_hists(I2_hsv2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
vis_pair(I2,I2_hsv2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# YCbCr Space

In [103]:
vis_hists(I2_ybr[...,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [104]:
Y,Cb,Cr = np.split(I2_ybr, indices_or_sections=3,axis=-1)
temp3 = Y.copy()

In [105]:
arr_info(temp3)

((2678, 4480, 1), dtype('float64'), 16.195811764705883, 154.15894117647056)

In [106]:
arr_info((235/np.log2(156))*np.log2(1.0 + temp3))

((2678, 4480, 1), dtype('float64'), 132.37931423936996, 234.74842701244776)

In [107]:
Y_e = (235/np.log2(156))*np.log2(1.0 + temp3)

In [108]:
arr_info(Y_e)

((2678, 4480, 1), dtype('float64'), 132.37931423936996, 234.74842701244776)

In [109]:
I2_ybr_e = (np.dstack((Y_e,Cb,Cr)))

In [110]:
arr_info(I2_ybr_e[...,2])

((2678, 4480), dtype('float64'), 97.1258274509804, 133.12773333333334)

In [111]:
def simple_linear(I, A = 1.0, B = 0.0):
    '''
    simpleLinear(I, A = 1.0, B = 0.0): return a linearly transformed image with the 
    provided constants A and B. 
    '''
    Tr = lambda x: A*x + B
    J = Tr(I.astype('float'))
    return np.clip(J, 0, 255).astype('uint8')

In [112]:
I2_ybr2rgb = color.ycbcr2rgb(I2_ybr_e)
J2 = I2_ybr2rgb.copy()

In [113]:
J_normed = simple_linear(J2, A = 255.0/(J2.max()-J2.min()), B = -255*J2.min()/(J2.max()-J2.min()))

In [114]:
arr_info(I2_ybr2rgb[...,])

((2678, 4480, 3), dtype('float64'), 0.5305182761400187, 1.1500473945736116)

In [115]:
vis_hists(I2_ybr2rgb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [116]:
vis_hists(J_normed)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:
vis_pair(I2,J_normed)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
f, ax = plt.subplots(1,4,figsize=(12,6),sharex=True,sharey=True)
ax[0].imshow(I2)
ax[0].set_title('Original')
ax[1].imshow(I2_lab2rgb)
ax[1].set_title('Lab Space')
ax[2].imshow(I2_hsv2rgb)
ax[2].set_title('HSV Space')
ax[3].imshow(J_normed)
ax[3].set_title('YCbCr Space')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'YCbCr Space')

## 3. Histogram Equalization

Again you can use `vis_hists`, `vis_pair`, `vis_[rgb,lab,hsv,ybr]_cube` to visualize results.

- **Histogram-equalize an image and show** the resulting change in the image (as in, "Before" and "After" with `vis_pair`).
- **Explain how**, with written description, the figures you generate do or do not reflect good contrast enhancement. Are there artifacts in the corrected image that make it not an "improvement" in your eyes? 
- Try with two alternative colorspaces: **Perform the histogram equalization on only the Luminance/Y/Value** dimension after conversion into  [Lab](../Color/color_Lab.ipynb)/[YCbCr](../Color/color_YCbCr.ipynb)/[HSV](../Color/color_HSV.ipynb) space, and **show the results**. 
- **Explain your results**.

In [119]:
def make_scurve(xc, s):
    return lambda x: 1/(1+np.exp(-(s*(x-xc))))

In [120]:
x = np.arange(0,1,1/100)
# f = lambda x: 1/(1+np.exp(-(10*(x-.5))))
f = make_scurve(.5, 10)

In [121]:
# I2 = plt.imread('../dip_pics/dark_nature.jpg')

In [122]:
Jhsv = color.rgb2hsv(I2)
Jlab = color.rgb2lab(I2)

In [123]:
vis_hists(Jhsv[...,2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [124]:
vis_hists(Jlab[...,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [125]:
L,A,B = np.split(Jlab, indices_or_sections=3,axis=-1)

In [126]:
arr_info(Jlab[...,0])

((2678, 4480), dtype('float64'), 0.039573654069084085, 66.6368544813074)

In [127]:
H,S,V = np.split(Jhsv, indices_or_sections=3,axis=-1)

In [128]:
arr_info(Jhsv[...,2])

((2678, 4480), dtype('float64'), 0.00392156862745098, 0.7607843137254902)

## HSV Space 

In [129]:
vis_hists(I2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
x

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99])

In [131]:
bins=np.arange(257)
freq, bins = np.histogram((255*Jhsv[...,2]).astype('uint8').ravel(), bins=bins) # normalize the range of V values from 0 to 1 float back to 0 to 255 integers and calcualte the cdf using that
freq = freq/freq.max()
cdf = np.cumsum(freq)
cdf = cdf/cdf.max()
func = interp1d(bins[:-1], cdf, fill_value='extrapolate') 

In [132]:
f = plt.figure(figsize=(4,3))
plt.bar(bins[:-1], freq, width=.5) # width to keep the bars skinny enough.
plt.plot(bins[:-1], cdf, 'b-', label='$CDF$');
plt.plot(x*255, make_scurve(.5, 15)(x), 'r-', label='S-curve');
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [133]:
func = interp1d(bins[:-1], cdf, fill_value='extrapolate') 

In [134]:
V_eqt = func((255*V).astype('uint8'))
arr_info(V_eqt)

((2678, 4480, 1), dtype('float64'), 7.501600341406169e-07, 1.0)

In [135]:
Jhsv_eq = np.dstack((H,S,V_eqt))

In [136]:
arr_info(Jhsv_eq[...,2])

((2678, 4480), dtype('float64'), 7.501600341406169e-07, 1.0)

In [137]:
Jhsv_eqt = (255*color.hsv2rgb(Jhsv_eq)).astype('uint8')

In [138]:
vis_hists(Jhsv_eqt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [139]:
vis_pair(I2,Jhsv_eqt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Lab Space 

In [140]:
bins2=np.arange(101)
freq2, bins2 = np.histogram((Jlab[...,0]).astype('uint8').ravel(), bins=bins2) # normalize the range of V values from 0 to 1 float back to 0 to 255 integers and calcualte the cdf using that
freq2 = freq2/freq2.max()
cdf2 = np.cumsum(freq2)
cdf2 = cdf2/cdf2.max()
func2 = interp1d(bins2[:-1], cdf2, fill_value='extrapolate') 

In [141]:
f2 = plt.figure(figsize=(4,3))
plt.bar(bins2[:-1], freq2, width=.5) # width to keep the bars skinny enough.
plt.plot(bins2[:-1], cdf2, 'b-', label='$CDF$');
plt.plot(x*100, make_scurve(.5, 15)(x), 'r-', label='S-curve');
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [142]:
func2 = interp1d(bins2[:-1], cdf2, fill_value='extrapolate') 

In [143]:
L_eqt = 100*func2(L.astype('uint8')) # proper range from 0 to 100, here is 0 to 1 so normalize it back

In [144]:
Jlab_eq = np.dstack((L_eqt,A,B))

In [145]:
arr_info(Jlab_eq[...,0])

((2678, 4480), dtype('float64'), 3.5243185212845405, 100.0)

In [146]:
vis_hists(Jlab_eq[...,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
Jlab_eqt = (255*color.lab2rgb(Jlab_eq)).astype('uint8')

In [149]:
vis_hists(Jlab_eqt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [150]:
vis_pair(I2,Jlab_eqt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [151]:
f, ax = plt.subplots(1,3,figsize=(12,6),sharex=True,sharey=True)
ax[0].imshow(I2)
ax[0].set_title('Original')
ax[1].imshow(Jhsv_eqt)
ax[1].set_title('Histeq through HSV Space')
ax[2].imshow(Jlab_eqt)
ax[2].set_title('Histeq through Lab Space')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Histeq through Lab Space')